# Second assignment of Research Track II

## Jupyter notebook of the node user_interface

With this implementation you will be able control the behavior of the robot within the Gazebo environment. In addition, you will have much more functionalities to control the simulation thanks to the presence of widgets and the Matplotlib library. 

In particular the new functionalities, implemented thanks to these tools, are now:
- the ability to start/stop the robot, that is regulated with two buttons;
- the ability of changing the velocities, both linear and angular, using two sliders;
- the ability of directly teleoperate the movements of the robot in each direction (plus the stop), with 5 buttons.

Moreover, you have some information, concerning the simulation, that are graphically displayed, including:
- a line plot that displays cmd_vel vs actual velocity (for both linear and angular velocities)
- a bar plot that allows to see the number of reached and canceled target positions during the execution of the simulation
- a hist plot that shows the time required to reach each target position
- an xy graph that shows the robot's position and orientation during the execution of the simulation

## How to run the code

After cloning the repository in a ROS workspace, you'll need two different terminals.

In the first terminals you need to type the following instruction:
- in your ROS workspace you need to build the package that you have just cloned using the command 

`catkin_make`
- then you need to run the

`roscore &`
- finally, you need to go inside the package that you have cloned and starts the simulation by typing

`roslaunch rt2_assignment1 notebook.launch`

In the second terminal you need to start the Jupyter notebook by typing

`jupyter notebook --allow-root --ip 0.0.0.0`


Then open the file **user_interface_notebook.ipynb** and run all the cells (the ones that you need) to be able to directly interact with the Gazebo simulation environment.


Advice: if you have not already done so, I recommend that you activate the nbextention *execute time*, to make sure that a cell has actually been run. 

## Headers and initialization of the node

This notebook, as it happens for the user_interface node, communicates with the other three nodes that are the:
- go_to_point; 
- position_service;
- state_machine.

Here are listed the headers that are needed for our purpose:

In [1]:
import rospy
import time
import math
from rt2_assignment1.srv import Command
from nav_msgs.msg import Odometry
import actionlib
import actionlib.msg
import rt2_assignment1.msg
from rt2_assignment1.msg import TargetActionResult

import matplotlib.pyplot as plt
import numpy as np
import jupyros as jr
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, FloatSlider
from IPython.display import display
%matplotlib widget

Now we initialize the node:

In [3]:
# initialization of the node
rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
    
# action client to cancel the goal 
act_c = actionlib.SimpleActionClient("go_to_point",rt2_assignment1.msg.TargetAction)
act_c.wait_for_server()

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

### Starting/stopping the robot using buttons

With this implementation, you will have two different buttons, that allow to interact directly with the simulation, making the robot starts and stops while the it is moving to reach its random position target within the simulation environment.
Each of the button has a callback function associated, so anytime one of the button is pressed by the user, the corresponding callback function will be executed.

In [4]:
start_button = widgets.Button(description='START', layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='limegreen')) 
stop_button = widgets.Button(description='STOP', layout=widgets.Layout(widht='auto',grid_area='b2'),
                style=widgets.ButtonStyle(button_color='firebrick')) 

In [18]:
output = widgets.Output()

start_time = 0
stop_time = 0

def start_on(b):
    global start_time
    # start the robot behavior
    ui_client("start")
    start_time=time.time()
        
    with output:
        print("The robot has been started")


def stop_on(b):
    global stop_time
    # stop the robot behavior
    act_c.cancel_all_goals()
    ui_client("stop")
    stop_time=time.time()
        
    with output:
        print("The robot has been stopped")
    
    
start_button.on_click(start_on) # on_click method for the start button
stop_button.on_click(stop_on) # on_click method for the stop button
display(output) # display the message

Output()

In [6]:
# arrange the buttons neatly
widgets.GridBox(children=[start_button,stop_button],
                layout=widgets.Layout(width='60%',grid_template_rows='auto auto',
                grid_template_columns='30% 20% 30%',
                grid_template_areas=''' "b1 . b2" '''))

GridBox(children=(Button(description='START', layout=Layout(grid_area='b1'), style=ButtonStyle(button_color='l…

### Changing the linear and the angular velocity using sliders

With this implementation you will have two different sliders, that allow to change the linear and the angular velocity while the simulation is running. 
In particular, the user can select velocities between -1 and 3 m/s.  

In [7]:
lin_vel_init = 0.5
lin_vel_slider = widgets.FloatSlider(description='Linear velocity', value=lin_vel_init, min=-1.0, max=1.5)
lin_vel_slider.style.handle_color = 'blue'

ang_vel_init = 0.5
ang_vel_slider = widgets.FloatSlider(description='Angular velocity', value=ang_vel_init, min=-1.0, max=1.5)
ang_vel_slider.style.handle_color = 'blue'

In [8]:
twist_msg = Twist()

def lin_vel_change(change): 
    global twist_msg
    twist_msg.linear.x = change['new']
    pub.publish(twist_msg)
    
def ang_vel_change(change):
    global twist_msg
    twist_msg.angular.z = change['new']
    pub.publish(twist_msg)
    
lin_vel_slider.observe(lin_vel_change, names='value')
ang_vel_slider.observe(ang_vel_change, names='value')

In [9]:
widgets.HBox([widgets.VBox([lin_vel_slider, ang_vel_slider])])

### Teleoperate the robot using buttons

With this implementation you will have 5 different buttons. 
Four of them are use to directly teleoperate the robot within the simulation and are the following ones:
- ↑ to go forward
- ↓ to go backward
- ← to turn left
- → to turn right
The other button is:
- STOP to stop the robot

In [10]:
go_forward_button = widgets.Button(description='\u2191',layout=Layout(width='auto', grid_area='b1'),
style=ButtonStyle(button_color='lavender'))
go_backward_button = widgets.Button(description='\u2193',layout=Layout(width='auto', grid_area='b2'),
style=ButtonStyle(button_color='lavender'))
turn_left_button = widgets.Button(description='\u2190',layout=Layout(width='auto', grid_area='b3'),
style=ButtonStyle(button_color='lavender'))
turn_right_button = widgets.Button(description='\u2192',layout=Layout(width='auto', grid_area='b4'),
style=ButtonStyle(button_color='lavender'))
stop_button = widgets.Button(description='STOP',layout=Layout(width='auto', align="center", grid_area='b5'),
style=ButtonStyle(button_color='firebrick'))

In [11]:
twist_msg = Twist()

def go_forward(click):
    global twist_msg
    twist_msg.linear.x = 1.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
def go_backward(click):
    global twist_msg
    twist_msg.linear.x = - 1.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
def turn_left(click):
    global twist_msg
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = -1.0
    pub.publish(twist_msg)
    
def turn_right(click):
    global twist_msg
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 1.0
    pub.publish(twist_msg)
    
def stop_behavior(behavior):
    act_c.cancel_all_goals()
    ui_client("stop")
    global twist_msg
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 0.0
    pub.publish(twist_msg)
    
go_forward_button.on_click(go_forward)
go_backward_button.on_click(go_backward)
turn_left_button.on_click(turn_left)
turn_right_button.on_click(turn_right)
stop_button.on_click(stop_behavior)

In [12]:
widgets.GridBox(children=[go_forward_button, go_backward_button, turn_left_button, turn_right_button ,stop_button],
                layout=widgets.Layout(width='75%',grid_template_rows='auto auto auto',
                grid_template_columns='15% 15% 15% 20% 35%',
                grid_template_areas='''
                ". b1 . . ."
                "b3 b2 b4 . ."
                " . . . . b5"
                ''')
               )

GridBox(children=(Button(description='↑', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyle(butto…

## Displaying graphical information

### Line plot: cmd_vel vs actual velocity

The following plots show the comparison between the requested velocity and the actual velocity, respectively for the linear and the angular velocities.

In [13]:
fig_vel, (ax_lin_vel, ax_ang_vel) = plt.subplots(1,2)

lin_vel_cmd, = ax_lin_vel.plot([], [], color='magenta', label = 'lin_vel_cmd')
lin_vel_odom, = ax_lin_vel.plot([], [], color='navy', label = 'lin_vel_odom')
ang_vel_cmd, = ax_ang_vel.plot([], [], color='mediumturquoise', label = 'ang_vel_cmd')
ang_vel_odom, = ax_ang_vel.plot([], [], color='olivedrab', label = 'ang_vel_odom')

lin_cmd_data = []
lin_odom_data = []
ang_cmd_data = []
ang_odom_data = []

ax_lin_vel.set_title('Linear velocity: cmd_vel vs odom')
ax_lin_vel.set(xlabel = 'Time [s]', ylabel = 'Linear velocity [m/s]')
ax_ang_vel.set_title('Angular velocity: cmd_vel vs odom')
ax_ang_vel.set(xlabel = 'Time [s]', ylabel = 'Angular velocity [rad/s]')

ax_lin_vel.legend()
ax_ang_vel.legend()

t = time.time()
time_vel = list(range(0, 10))

def init_vel():
    lin_vel_cmd.set_data([], [])
    lin_vel_odom.set_data([], [])
    ang_vel_cmd.set_data([], [])
    ang_vel_odom.set_data([], [])
    return (lin_vel_cmd, lin_vel_odom, ang_vel_cmd, ang_vel_odom)

def cmd_vel_callback(msg):
    if len(lin_cmd_data) > 20:
        lin_cmd_data.pop(0)
    if len(ang_cmd_data) > 20:
        ang_cmd_data.pop(0)
    lin_cmd_data.append(msg.linear.x)
    ang_cmd_data.append(msg.angular.z)
    
def odom_vel_callback(msg):
    if len(time_vel) > 10:
        time_vel.pop(0)
    if len(lin_odom_data) > 20:
        lin_odom_data.pop(0)
    if len(ang_odom_data) > 20:
        ang_odom_data.pop(0)
    time_vel.append(time.time()-t)
    lin_odom_data.append(msg.twist.twist.linear.x)
    ang_odom_data.append(msg.twist.twist.angular.z)
    
def animate_vel(i):
    lin_vel_cmd.set_data(time_vel, lin_cmd_data)
    lin_vel_odom.set_data(time_vel, lin_odom_data)
    ang_vel_cmd.set_data(time_vel, ang_cmd_data)
    ang_vel_odom.set_data(time_vel, ang_odom_data)
    return (lin_vel_cmd, lin_vel_odom, ang_vel_cmd, ang_vel_odom)

jr.subscribe('/cmd_vel', Twist, cmd_vel_callback)
jr.subscribe('/odom', Odometry, odom_vel_callback)

anim_vel = animation.FuncAnimation(fig_vel, animate_vel, init_func=init_vel, frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Bar plot: reached and canceled targets

With this plot you're going to visualize how many target positions have been reached by the robot within the simulation and how many targets have been canceled by the user.
In addition, the method FuncAnimation is used: it makes an animation by repeatedly calling a function, that in this case is the animate_barplot(animation), that updates the animation every time it is called.

In [14]:
fig_barplot, ax_barplot = plt.subplots()

ax_barplot.set_title('Number of reached and canceled target positions')
ax_barplot.set(ylabel = 'Number of targets', ylim = (0,30))

targets = np.array(['Reached','Canceled'])
barplot = ax_barplot.bar(targets, [0,0], width = 0.5, color = 'teal')

reached = 0
canceled = 0 
num = [0,0]

def init_barplot():
    ax_barplot.bar(targets, [0,0], width = 0.5, color = 'teal')
    return (ax_barplot)

def target_callback(msg):
    global reached, canceled
    if(msg.data == True):
        reached += 1
    else:
        canceled += 1
    num = (reached, canceled)
             
def animate_barplot(i):
    ax_barplot.bar(targets, num, width = 0.5, color = 'teal')
    return (ax_barplot)

jr.subscribe('/go_to_point/result', TargetActionResult, target_callback)

anim_bar = animation.FuncAnimation(fig_barplot, animate_barplot, init_func = init_barplot, frames=100, interval=10, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Hist plot: required time to reach the targets

With this plot you're going to visualize the time required, in seconds, to reach each target within the simulation.
In addition, the method FuncAnimation is used: it makes an animation by repeatedly calling a function, that in this case is the animate_histplot, that updates the animation every time it is called.

In [15]:
fig_histplot, ax_histplot = plt.subplots()

ax_histplot.set_title('Time needed to reach the target')
ax_histplot.set(xlabel = 'Time [s]', ylabel = 'Target', xlim = (0,60), ylim = (0,10))

histplot = ax_histplot.hist([], bins = 100, color = 'orange')

t = time.time()
time_required = [] 

def init_histplot():
    ax_histplot.hist([], bins = 100, color = 'orange')
    return (ax_histplot)

#def time_callback(msg):
#    global t, time_required
#    time_required.append(time.time() - t)
    
def animate_histplot(i):
    ax_histplot.hist([], bins = 100, color = 'orange')
    return (ax_histplot)


anim_hist = animation.FuncAnimation(fig_histplot, animate_histplot, init_func = init_histplot, frames=100, interval=10, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### xy graph: robot's position and orientation

With this graph you are able to visualize the trajectory that the robot is following in order to reach the target position in the simulation environment. So, you are going to see both the real-time position in the space and also the orientation of the robot. The orientation is highlighted by an arrow.  

In [16]:
fig_pos, ax_pos = plt.subplots()

pos_plt, = ax_pos.plot([], [], 'orchid')
arrow_plt, = ax_pos.plot([], [], 'darkviolet', linewidth = 3)

ax_pos.set_title('Position and orientation of the robot')
ax_pos.set(xlabel = 'x', ylabel = 'y', xlim = (-7,7) , ylim = (-7,7))

arrow_mod = 0.8
x_pos_data = []
y_pos_data = []

def init_pos():
    pos_plt.set_data([], [])
    arrow_plt.set_data([], [])
    return (pos_plt, arrow_plt)

def odom_callback_pos(msg):
    x_pos_data.append(msg.pose.pose.position.x)
    y_pos_data.append(msg.pose.pose.position.y)
    quaternion = (msg.pose.pose.orientation.x,
                  msg.pose.pose.orientation.y,
                  msg.pose.pose.orientation.z,
                  msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    angle = euler[2]

def animate_pos(i):
    pos_plt.set_data(x_pos_data, y_pos_data)
    x_orient_data = x_pos_data[len(x_pos_data)-1] + arrow_mod*math.cos(angle)
    y_orient_data = y_pos_data[len(y_pos_data)-1] + arrow_mod*math.sin(angle)
    arrow_plt.set_data(x_orient_data, y_orient_data) 
    return (pos_plt, arrow_plt)

jr.subscribe('/odom', Odometry, odom_callback_pos)

anim_pos = animation.FuncAnimation(fig_pos, animate_pos, init_func = init_pos, frames = 100, interval = 20, blit = True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now
